In [12]:
import pyorc
# import cartopy
import xarray as xr
# import cartopy.crs as ccrs
import pandas as pd
from matplotlib import patches
import matplotlib.pyplot as plt
# import cartopy.io.img_tiles as cimgt
# from matplotlib.colors import Normalize
# from dask.diagnostics import ProgressBar
import glob
import cv2

In [3]:
months = ['January', 'February', 'March', 'April', 'May', 'June']
videos = []
for month in months:
    for video in sorted(glob.glob("*.mp4", root_dir=f"{month}/videos/")):
        videos.append(
            {
                'video': f'{month}/videos/{video}',
                'name': video.split('.')[0]
            }
        )
videos = ['March/videos/video_capture_2024-03-01_07-42-19.mp4']

In [17]:
# cap = cv2.VideoCapture(videos[0])

# while cap.isOpened():
#     ret, frame = cap.read()
#     cv2.imwrite('anchors.jpg', frame)

#     break

In [ ]:
# current_file = videos.pop()
current_file = videos.pop(0)
video_file = current_file['video']
video_name = current_file['name']
video = pyorc.api.video.Video(video_file, start_frame=0, end_frame=125)
frame = video.get_frame(0, method="rgb")

In [ ]:
plt.plot(1613, 40, "rx", markersize=1)
plt.imshow(frame)

In [ ]:
gcps = dict(
    src=[
        [2434, 1397], #Blue
        [2304, 1001], #Green
        [525, 879],   #Orange
        [77, 1779]    #Purple
    ]
)

In [ ]:
plt.imshow(frame)
plt.plot(*zip(*gcps["src"]), "rx", markersize=20, label="Control points")
plt.legend()
# plt.savefig(f"Camera_test/Control/{video_name}_control.png", bbox_inches="tight", dpi=600)

In [ ]:
gcps["dst"] = [
    [13.633,6.171],  # Blue
    [15.307,9.675],   # Green
    [3.412,15.957],  # Orage
    [5.747,4.855]    # Purple    
]

gcps["z_0"] = 0.00

In [ ]:
height, width = frame.shape[0:2]

In [ ]:
cam_config = pyorc.CameraConfig(height=height, width=width, gcps=gcps)

In [ ]:
corners = [
    [2559, 1919],
    [1752, 680],
    [940, 680],
    [200, 1919]
]
cam_config.set_bbox_from_corners(corners)
cam_config.resolution = 0.01
cam_config.window_size = 25

In [ ]:
plt.imshow(frame)
plt.plot(*zip(*gcps["src"]), "rx", markersize=20, label="Control points")
plt.plot(*zip(*corners), "co", label="Corners of AOI")
plt.legend()
# plt.savefig(f"Camera_test/Corners/{video_name}_corners.png", bbox_inches="tight", dpi=600)

In [ ]:
ax2 = plt.axes()
ax2.imshow(frame)
cam_config.plot(ax=ax2, camera=True)
# plt.savefig(f"Camera_test/Cam_config/{video_name}_cam_config.png", bbox_inches="tight", dpi=600)

In [ ]:
# cam_config.to_file(f"Cam_config/{video_name}_cam-config-UWRL.json")

In [ ]:
# cam_config = pyorc.load_camera_config(f"Cam_config/{video_name}_cam-config-UWRL.json")
# cam_config = pyorc.load_camera_config("cam-config-UWRL.json")

In [ ]:
stabilize = [
    [2559, 1919],
    [1752, 680],
    [940, 680],
    [200, 1919]
]
video = pyorc.Video(
    video_file,
    camera_config=cam_config,
    start_frame=0,
    end_frame=125,
    stabilize=stabilize,
    h_a=0.
)
# video

In [ ]:
patch_kwargs = {
    "alpha": 0.5,
    "zorder": 2,
    "edgecolor": "w",
    "label": "Area of interest",
}
f, ax = plt.subplots(1, 1, figsize=(10, 6))

frame = video.get_frame(0, method="rgb")
# plot frame on a notebook-style window
ax.imshow(frame)
# add the polygon to the axes
patch = patches.Polygon(
    stabilize,
    **patch_kwargs
)
p = ax.add_patch(patch)

In [ ]:
plt.imshow(frame)

In [ ]:
da = video.get_frames()
da[0].frames.plot(cmap="gray", ax=ax)

In [ ]:
da_norm = da.frames.normalize()
da_norm[0].frames.plot(cmap="gray")

In [ ]:
da_norm_proj = da_norm.frames.project()
da_norm_proj[0].frames.plot(cmap="gray")

In [ ]:
# extract frames again, but now with rgb
da_rgb = video.get_frames(method="rgb")
# project the rgb frames, same as before
da_rgb_proj = da_rgb.frames.project()

In [ ]:
# again the rgb frame first. But now we use the "geographical" mode to plot on a map
p = da_rgb_proj[0].frames.plot(mode="geographical")

#...and then masked velocimetry again, but also geographical
ds_mean_mask2.velocimetry.plot(
    ax=p.axes,
    mode="geographical",
    alpha=0.4,
    cmap="rainbow",
    scale=20,
    width=0.0015,
    norm=Normalize(vmax=0.6, clip=False),
    add_colorbar=True
)

# for fun, let's also add a satellite background from cartopy
import cartopy.io.img_tiles as cimgt
import cartopy.crs as ccrs
tiles = cimgt.GoogleTiles(style="satellite")
p.axes.add_image(tiles, 19)
# zoom out a little bit so that we can actually see a bit
p.axes.set_extent([
    da_rgb_proj.lon.min() - 0.00005,
    da_rgb_proj.lon.max() + 0.00005,
    da_rgb_proj.lat.min() - 0.00005,
    da_rgb_proj.lat.max() + 0.00005],
    crs=ccrs.PlateCarree()
)
